# Домашнее задание0

## URL:

- https://www.dns-shop.ru/product/4e489f4faf9d3330/processor-intel-celeron-g3930-oem/characteristics/
-Получить данные о продукте"Характеристики Процессор Intel Celeron G3930 OEM"


- https://www.dns-shop.ru/product/84be09f917d53332/termopasta-arctic-cooling-mx-4-2019/characteristics/
-Получить данные о продукте"Характеристики Термопаста Arctic Cooling MX-4 (2019)"


### EXCEL легко получить данные


In [26]:
import requests
import bs4


#要抓取的目标页码地址
url = 'https://www.dns-shop.ru/product/4e489f4faf9d3330/processor-intel-celeron-g3930-oem/characteristics/' 
headers={'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'}

#抓取页码内容，返回响应对象
response = requests.get(url,headers=headers)

#查看响应状态码
status_code = response.status_code

#使用BeautifulSoup解析代码,并锁定页码指定标签内容
content = bs4.BeautifulSoup(response.content.decode("utf-8"), "lxml")
element = content.find_all("tr")

print(status_code)
print(element)

200
[<tr><td class="table-part" colspan="2">Заводские данные</td></tr>, <tr><td><div class="dots"><span> Гарантия </span></div></td><td><div id="pcv-9VsHqT"> 12 мес.<!-- --></div></td></tr>, <tr><td><div class="dots"><span> Страна-производитель <span data-characteristic-hint="Страна производитель может отличаться от заявленной" data-characteristic-title="Страна-производитель"></span></span></div></td><td><div id="pcv-LctF5x"> Вьетнам<!-- --></div></td></tr>, <tr><td class="table-part" colspan="2">Общие параметры</td></tr>, <tr><td><div class="dots"><span> Модель <span id="as-ccf6e64a1109c41abbfb7d9bb351d667"></span></span></div></td><td><div id="pcv-yJmhDT"> Intel Celeron G3930<!-- --></div></td></tr>, <tr><td><div class="dots"><span> Поколение процессоров <span id="as-c3da72ebaf206a23ddd5800b365371c0"></span></span></div></td><td><div id="pcv-Y67x52"> Intel 6-е поколение<!-- --></div></td></tr>, <tr><td><div class="dots"><span> Код производителя <span id="as-23e1482e02ef65a22a692b1918

In [27]:
#要抓取的目标页码地址
url = 'https://www.dns-shop.ru/product/84be09f917d53332/termopasta-arctic-cooling-mx-4-2019/characteristics/' 
headers={'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'}

#抓取页码内容，返回响应对象
response = requests.get(url,headers=headers)

#查看响应状态码
status_code = response.status_code

#使用BeautifulSoup解析代码,并锁定页码指定标签内容
content = bs4.BeautifulSoup(response.content.decode("utf-8"), "lxml")
element = content.find_all("tr")

print(status_code)
print(element)

200
[<tr><td class="table-part" colspan="2">Заводские данные</td></tr>, <tr><td><div class="dots"><span> Гарантия </span></div></td><td><div id="pcv-3xOYzr"> 12 мес.<!-- --></div></td></tr>, <tr><td><div class="dots"><span> Страна-производитель <span data-characteristic-hint="Страна производитель может отличаться от заявленной" data-characteristic-title="Страна-производитель"></span></span></div></td><td><div id="pcv-s3atCA"> Китай<!-- --></div></td></tr>, <tr><td class="table-part" colspan="2">Общие параметры</td></tr>, <tr><td><div class="dots"><span> Модель <span id="as-3d403bf45a16824ad14a7e6d9cf52b37"></span></span></div></td><td><div id="pcv-AiN0a2"> Arctic Cooling MX-4 (2019)<!-- --></div></td></tr>, <tr><td class="table-part" colspan="2">Характеристики термоинтерфейса</td></tr>, <tr><td><div class="dots"><span> Теплопроводность <span id="as-7912287fcec38e0fbde738a39e2f066a"></span></span></div></td><td><div id="pcv-gLToex"> 8.5 Вт/мК<!-- --></div></td></tr>, <tr><td><div class=